# XtEHR Logical model processing

* ophalen XtEHR Logical Models
* consistente layout met zibs
* klaar voor een mapping - zie mapping Notebook

## Relevante kern content

NL kernset:
- Behandelaanwijzing

ePS required:
- Medicatie
- Diagnose/ Probleemlijst(actief)
- Verrichtingen(<6mnd, ≥ 6mnd)
- Allergieën en intoleranties + - - 
- Alerts
- Hulpmiddel
- Patiënt

ePS recommended + optional:
- Vaccinations
- Implants
- Treatment restrictions
- Vital signs
- History of illness
- Pregnancy
- Social history
- Functional status
- Results
- Plan of Care
- Encounters

# Imports

In [93]:
import pandas as pd
import numpy as np
import requests
import json
import os
from io import StringIO


# Download JSON

* Dit hoeft maar een keer te gebeuren, bij nieuwe mappings download de JSON die nodig is. 
* Kan ook door directe download 'raw' JSON.
* Dit zijn de XtEHR Logical Models

In [94]:
xtehrlist = ['Location','DispenseDecline','Dosaging','Medication','MedicationDispense','MedicationPrescription']
xtehrbase = 'https://build.fhir.org/ig/Xt-EHR/xt-ehr-common/StructureDefinition-EHDS'
# res = 'AlertFlag'
os.makedirs(os.path.join('xtehr', 'downloads'), exist_ok=True)

for res in xtehrlist:
    print(res)
    url = xtehrbase + res + '.json'
    response = requests.get(url)
    jsondata = response.json()
    with open(os.path.join('xtehr', 'downloads', res + '.json'), 'w') as jsonfile:
        json.dump(jsondata, jsonfile, indent=4)

Location
DispenseDecline
Dosaging
Medication
MedicationDispense
MedicationPrescription


# List downloaded files

* Define the base directory for downloads.
* Construct the folder path for the downloaded files.
* List all files in the specified folder.

In [95]:
base = 'xtehr'
folder = os.path.join(base, 'downloads')
files = os.listdir(folder)
files

['DispenseDecline.json',
 'Dosaging.json',
 'Location.json',
 'Medication.json',
 'MedicationDispense.json',
 'MedicationPrescription.json']

# Kies en lees het te verwerken Logical Model

* Toon en verifieer dat de uitvoer er is

In [ ]:
base = 'xtehr'
file = files[0]  # Selecting the n-th file in the list (run through them one by one)
url = os.path.join(base, 'downloads', file)
with open(url) as jsonfile:
    xlm = json.load(jsonfile)['snapshot']['element']
dfx = pd.DataFrame(xlm)
dfx

IndexError: list index out of range

# Opschonen uitvoer zodat zib en xtehr consistent zijn

* een card. veld met n..m
* type (is een JSON object) omzetten naar tekst
* alleen nodige kolommen houden

In [ ]:
if 'card.' not in dfx.columns:
    dfx.insert(2, 'card.', pd.NA)

if 'min' in dfx.columns and 'max' in dfx.columns:
    # Use nullable Int for min when possible, then stringify
    try:
        dfx['card.'] = dfx['min'].astype('Int64').astype(str).replace('<NA>', '') + '..' + dfx['max'].astype(str)
    except Exception:
        dfx['card.'] = dfx['min'].astype(str) + '..' + dfx['max'].astype(str)

# --- type normalization ---
if 'type' in dfx.columns:
    dfx['type'] = dfx['type'].apply(
        lambda v: (
            # list → first element
            (isinstance(v, list) and (v[0].get('code') if (len(v) and isinstance(v[0], dict)) else (v[0] if len(v) else np.nan)))
            # dict → code
            or (v.get('code') if isinstance(v, dict) else None)
            # anything else (string/NaN) → as is
            or v
        )
    )
    # If it's a URI or path-like, take the last segment
    dfx['type'] = dfx['type'].apply(lambda s: s.split('/')[-1] if isinstance(s, str) else s)

# --- binding synthesis (if needed) ---
if 'binding' not in dfx.columns:
    # prefer common subfields if present
    for pref in ['binding.valueSet', 'binding.description', 'binding.strength']:
        if pref in dfx.columns:
            dfx['binding'] = dfx[pref]
            break
    else:
        # any column starting with 'binding'
        bind_cols = [c for c in dfx.columns if c.lower().startswith('binding')]
        dfx['binding'] = dfx[bind_cols[0]] if bind_cols else pd.NA

# --- final column order without KeyError ---
wanted = ['id', 'path', 'short', 'definition', 'type', 'card.', 'binding']
dfx = dfx.reindex(columns=wanted, fill_value=pd.NA)

dfx

,id,path,short,definition,type,card.,binding
0,EHDSMedicationPrescription,EHDSMedicationPrescription,Medication prescription model,Logical model for medication prescription. A p...,NaN,0..*,NaN
1,EHDSMedicationPrescription.header,EHDSMedicationPrescription.header,Prescription header,Prescription header data elements,Base,1..1,NaN
2,EHDSMedicationPrescription.header.subject,EHDSMedicationPrescription.header.subject,The person for whom the medication is prescrib...,Patient/subject information,EHDSPatient,1..1,NaN
3,EHDSMedicationPrescription.header.identifier,EHDSMedicationPrescription.header.identifier,Business identifier(s) for the prescription. [...,Business identifier for the object,Identifier,0..*,NaN
4,EHDSMedicationPrescription.header.authorship,EHDSMedicationPrescription.header.authorship,Authorship,Resource authoring details,Base,1..*,NaN
5,EHDSMedicationPrescription.header.authorship.a...,EHDSMedicationPrescription.header.authorship.a...,"The prescriber, the person who made the prescr...",Author(s) by whom the resource was/were author...,EHDSHealthProfessional,1..1,NaN
6,EHDSMedicationPrescription.header.authorship.d...,EHDSMedicationPrescription.header.authorship.d...,Time of issuing (signing) the prescription by ...,Date and time of the issuing the document/reso...,dateTime,1..1,NaN
7,EHDSMedicationPrescription.header.lastUpdate,EHDSMedicationPrescription.header.lastUpdate,Date and time of the last update to the resource,Date and time of the last update to the docume...,dateTime,0..1,NaN
8,EHDSMedicationPrescription.header.status,EHDSMedicationPrescription.header.status,"Status of the prescription, this should not be...",Status of the resource,CodeableConcept,1..1,NaN
9,EHDSMedicationPrescription.header.statusReason[x],EHDSMedicationPrescription.header.statusReason[x],"Reason for the current status of prescription,...",Reason for the current status of the resource.,CodeableConcept,0..1,NaN


# Opslaan in Excel

In [ ]:
os.makedirs(os.path.join('xtehr', 'mappable'), exist_ok=True)
url = os.path.join(base, 'mappable', file.removesuffix(".json") + '.xlsx')
dfx.to_excel(url, index=False)